In [1]:
%load_ext autoreload
%autoreload 2

In [73]:
import pandas as pd 
import requests 
from bs4 import BeautifulSoup 
import re
from clean_data import *
from webscrap import *

import copy
#import dtale

### Pendiente: 
1) funcion que clasifica campo periodo, en fecha. 
2) limpiar periodos unidos. 
3) clasificar sexo de ministrxs

In [7]:
url_dic = {
    'pcm': 'https://es.wikipedia.org/wiki/Anexo:Presidentes_del_Consejo_de_Ministros_del_Per%C3%BA',
    'rree': 'https://es.wikipedia.org/wiki/Ministro_de_Relaciones_Exteriores_del_Per%C3%BA',
    'defensa': 'https://es.wikipedia.org/wiki/Ministro_de_Defensa_del_Per%C3%BA', # necesita limpiar encabezados
    'economia': 'https://es.wikipedia.org/wiki/Ministro_de_Econom%C3%ADa_y_Finanzas_del_Per%C3%BA',## necesita limpiar fechas
    'interior': 'https://es.wikipedia.org/wiki/Ministro_del_Interior_del_Per%C3%BA', ## necesita limpiar fechas
    'justicia': 'https://es.wikipedia.org/wiki/Ministro_de_Justicia_y_Derechos_Humanos_del_Per%C3%BA',
    'educacion': 'https://es.wikipedia.org/wiki/Ministro_de_Educaci%C3%B3n_del_Per%C3%BA',
    'salud': 'https://es.wikipedia.org/wiki/Ministro_de_Salud_del_Per%C3%BA',
    'agro': 'https://es.wikipedia.org/wiki/Ministro_de_Desarrollo_Agrario_y_Riego_del_Per%C3%BA',
    'trabajo': 'https://es.wikipedia.org/wiki/Ministro_de_Trabajo_y_Promoci%C3%B3n_del_Empleo_del_Per%C3%BA',
    'produccion': 'https://es.wikipedia.org/wiki/Ministro_de_la_Producci%C3%B3n_del_Per%C3%BA', # solo produccion
    'comercio': 'https://es.wikipedia.org/wiki/Ministro_de_Comercio_Exterior_y_Turismo_del_Per%C3%BA', 
    'energia': 'https://es.wikipedia.org/wiki/Ministro_de_Energ%C3%ADa_y_Minas_del_Per%C3%BA', 
    'transporte': 'https://es.wikipedia.org/wiki/Anexo:Ministros_de_Transportes_y_Comunicaciones_del_Per%C3%BA',
    'vivienda': 'https://es.wikipedia.org/wiki/Ministro_de_Vivienda,_Construcci%C3%B3n_y_Saneamiento_del_Per%C3%BA',
    'mujer': 'https://es.wikipedia.org/wiki/Ministerio_de_la_Mujer_y_Poblaciones_Vulnerables',
    'ambiente': 'https://es.wikipedia.org/wiki/Ministro_del_Ambiente_del_Per%C3%BA',
    'cultura': 'https://es.wikipedia.org/wiki/Ministro_de_Cultura_del_Per%C3%BA',
    'desarrollo': 'https://es.wikipedia.org/wiki/Ministro_de_Desarrollo_e_Inclusi%C3%B3n_Social_del_Per%C3%BA'
    
}

# nos dice cuantos 
url_ntable = {
    'pcm': 1,
    'rree': 1,
    'defensa': 1, # necesita limpiar encabezados
    'economia': 1,## necesita limpiar fechas
    'interior': 1, ## necesita limpiar fechas
    'justicia': 2, ## debe ser 2
    'educacion': 1,
    'salud': 1,
    'agro': 1,
    'trabajo': 2, ## debe ser 2, desde 2003
    'produccion': 2, ## desde 2002
    'comercio': 2, # desde el 2003, falta completar anios
    'energia': 1,
    'transporte': 1,
    'vivienda': 2, #desde el 2002, tiene que ser mastranst
    'mujer': 2 ,# homogenizar campos, completar tiene q ser 2
    'ambiente': 1,
    'cultura': 1,
    'desarrollo': 1
}

In [8]:
#df = get_df_from_soup(aa)
scrap = {}
for key, val in url_dic.items():
    scrap[key] = get_wiki_table(val, url_ntable[key])

3
6
1
2
5
13
2
1
1
5
2
3
1
1
2
3
1
1
1


In [42]:
d2 = copy.deepcopy(scrap)

In [43]:
#agregando una variable necesaria
d2['vivienda']['Gobierno'] = ''        


for key, val in d2.items():
    d2[key] = limpia_base(val)
    
#agregando fin necesario
d2['educacion']['fin'] = d2['educacion']['inicio'].shift(-1)
d2['educacion'] = d2['educacion'].iloc[d2['educacion'].index > 48]
d2['educacion'] = d2['educacion'].reset_index(drop = True)

periodos separados: pcm, rree, defensa, salud, vivienda, cultura  
periodos unidos: economia, interior, justicia, agro, trabajo, produccion, comercio, energia, transporte, mujer, ambiente, desarrollo  
sin fin, solo inicio: educacion

In [44]:
por_separar = ['economia', 'interior', 'justicia','agro', 'trabajo',
'produccion', 'comercio', 'energia', 'transporte', 'mujer', 'ambiente', 'desarrollo']

for key in d2.keys():
    if key in por_separar:
        #print(key)
        d2[key].dropna(subset=['Ministro'], inplace = True)
        d2[key]['Periodo'] = d2[key]['Periodo'].apply(lambda x: clean_dash(x))
        d2[key]['Periodo_sp'] = d2[key]['Periodo'].str.split('-')
        d2[key]['inicio'] = d2[key]['Periodo_sp'].apply(lambda x: x[0])
        d2[key]['fin'] = d2[key]['Periodo_sp'].apply(lambda x: x[1])
        

In [45]:
for key in d2.keys():
    print(key)
    d2[key] = d2[key][['Ministro', 'Presidente', 'inicio', 'fin']]
    d2[key].loc[len(d2[key]) - 1, "fin"] = "11 de julio de 2021"

pcm
rree
defensa
economia
interior
justicia
educacion
salud
agro
trabajo
produccion
comercio
energia
transporte
vivienda
mujer
ambiente
cultura
desarrollo


/Users/ccsuehara/opt/anaconda3/envs/dep/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/ccsuehara/opt/anaconda3/envs/dep/lib/python3.9/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [46]:
for key in d2.keys():
    print(key)
    d2[key] = get_fechas(d2[key], 'inicio')
    d2[key] = get_fechas(d2[key], 'fin')

pcm
rree
defensa
economia
interior
justicia
educacion
salud
agro
trabajo
produccion
comercio
energia
transporte
vivienda
mujer
ambiente
cultura
desarrollo


/Users/ccsuehara/GH_folders/apellidos/scripts/clean_data.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ex['f_' + str_var] = df_ex[str_var].apply(lambda x: get_day(x) + "/" +


In [87]:
nombres_hombres = set_nombres('../data/hombres.csv')
nombres_mujeres = set_nombres('../data/mujeres.csv')

In [88]:
for key in d2.keys():
    d2[key]['nombre'] = d2[key]['Ministro'].str.split(' ')
    d2[key]['nombre'] = d2[key]['nombre'].apply(lambda x: x[0].replace("é", "e").replace("á", "a").replace("í", "i").replace("ó", "o").replace("ú", "u"))
    d2[key]['genero'] = d2[key]['nombre'].apply(lambda x: pertenece_nombres(x,nombres_hombres, nombres_mujeres))

In [91]:
ministros = pd.DataFrame()
for key, val in d2.items():
    ministros = pd.concat([ministros,d2[key]])

In [92]:
ministros

,Ministro,Presidente,inicio,fin,f_inicio,f_fin,nombre,genero
0,Javier Pérez de Cuéllar de la Guerra,Valentín Paniagua,22 de noviembre de 2000,28 de julio del 2001,22/11/2000,28/7/2001,Javier,Hombre
1,Roberto Dañino Zapata,Alejandro Toledo,28 de julio de 2001,11 de julio de 2002,28/7/2001,11/7/2002,Roberto,Hombre
2,Luis Solari de la Fuente,Alejandro Toledo,12 de julio de 2002,23 de junio de 2003,12/7/2002,23/6/2003,Luis,Hombre
3,Beatriz Merino Lucero,Alejandro Toledo,23 de junio de 2003,12 de diciembre de 2003,23/6/2003,12/12/2003,Beatriz,Hombre
4,Carlos Ferrero Costa,Alejandro Toledo,15 de diciembre de 2003,15 de agosto de 2005,15/12/2003,15/8/2005,Carlos,Hombre
5,Pedro Pablo Kuczynski Godard,Alejandro Toledo,16 de agosto de 2005,28 de julio de 2006,16/8/2005,28/7/2006,Pedro,Hombre
6,Jorge del Castillo Gálvez,Alan García Pérez,28 de julio de 2006,10 de octubre de 2008,28/7/2006,10/10/2008,Jorge,Hombre
7,Yehude Simon Munaro,Alan García Pérez,14 de octubre de 2008,11 de julio de 2009,14/10/2008,11/7/2009,Yehude,no match
8,Javier Velásquez Quesquén,Alan García Pérez,11 de julio de 2009,13 de septiembre de 2010,11/7/2009,13/9/2010,Javier,Hombre
9,José Antonio Chang Escobedo,Alan García Pérez,14 de septiembre de 2010,18 de marzo de 2011,14/9/2010,18/3/2011,Jose,Hombre
